Written by:
 - Justin Tijunelis
 - Richard Lee
 - Erik Skoronski
 - QiFeng Li

# Make a parser

Parsers:
- Parser for concurrent session data
- Parser for httpperf-detailed output
- Parser for httperf summary output
- Parser for perform data (Justin)

2 - 3: Justin, Richard, Willie
4, 5, 6: Mix of everyone else, moreso Willie and Erik

Before Feb. 18, finish parsers and parts 2-3. 

Before Feb. 25, finish 4, 5, 6 and the document. 




# Imports

In [ ]:
import re
import numpy as np
import pandas as pd

# Concurrent Session Parsing

In [ ]:
# TODO: Delete this comment - Richard
# Parse histogram data for number of concurrent users
def parse_concurrent_session_data(file):
  df = pd.read_csv(file)
  # Note space in front of "frequency"...exact string: ' frequency of occurence'
  df = df.rename(columns={'# of concurrent sessions': 'num_sessions', ' frequency of occurence': 'frequency'})
  return df

df = parse_concurrent_session_data("Case1_Data/case1-concurrent-sessions-histogram.txt")
print(df.head(5))

# HTTPPerf Details Parsing

In [ ]:
# Someone do this - Willie
def parse_httperf_details(file):
  df = pd.read_csv(file)
  return df

df = parse_httperf_details("Case1_Data/case1-httperf-detailed-output.csv")
display(df)
df.info


# HTTPerf Summary Parsing

In [ ]:
# Someone do this - Erik
def parse_httperf_summary(file):
  if file == "case1-httperf-detailed-output.csv":
    pass
  else:
    print("hi")
    # Return dataframe?
df = parse_httperf_summary("Case1_Data/case1-httperf-summary-output.txt")
print(df)

# Perfmon Parsing

In [ ]:
def parse_perfmon(file):
  df = pd.read_csv(file, sep=',')
  df = df.rename(columns={
    '\PhysicalDisk(_Total)\% Disk Time': 'dis',
    '\Memory\Available Bytes': 'available_bytes',
    '\Memory\Cache Bytes': 'cache_bytes',
    '\Memory\Cache Faults/sec': 'cache_faults/sec',
    '\Memory\Page Faults/sec': 'page_faults/sec',
    '\Memory\Page Reads/sec': 'page_reads/sec',
    '\Memory\Page Writes/sec': 'page_writes/sec',
    '\Memory\Pages Input/sec': 'pages_input/sec',
    '\Memory\Pages Output/sec': 'pages_output/sec',
    '\Memory\Pool Nonpaged Bytes': ''
  })
  display(df)
  

parse_perfmon("./Case1_Data/case1-perfmon-data.csv")

# Case 1 Data

# Case 2 Data